In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
plt.style.use("dark_background")
plt.rcParams['figure.dpi'] = 150

In [4]:
bike_data = pd.read_csv("../data/processed_df.csv", index_col=0)
bike_data.head() # Load the processed bike data and inspect the first 5 rows

,start_id,end_id,start_time,duration,end_time,dist
1,103,37,47260920,360,47261280,1.458333
2,39,539,47260920,120,47261040,0.545517
3,785,785,47260920,300,47261220,0.000000
4,341,159,47260980,1800,47262780,1.092775
5,708,573,47260980,1080,47262060,3.607290


In [5]:
station_data = pd.read_csv("../data/santander_locations.csv")
station_data.head() # Load the station data and inspect the first 5 rows

,Station.Id,StationName,longitude,latitude,Easting,Northing
0,1,"River Street, Clerkenwell",-0.109971,51.5292,531202.520,182832.020
1,2,"Phillimore Gardens, Kensington",-0.197574,51.4996,525207.070,179391.860
2,3,"Christopher Street, Liverpool Street",-0.084606,51.5213,532984.810,182001.530
3,4,"St. Chad's Street, King's Cross",-0.120974,51.5301,530436.760,182911.990
4,5,"Sedding Street, Sloane Square",-0.156876,51.4931,528051.649,178742.097


In [9]:
bike_data = pd.read_csv("../data/processed_df.csv", index_col=0)
bike_data.head() # Load the processed bike data and inspect the first 5 rows

# Find minimum start time
x = bike_data.min()["start_time"]
t_min = (x // 86400) * 86400

# Substract t_min from start_time and end_time
bike_data["start_time"] = (bike_data["start_time"] - t_min) / 60
bike_data["end_time"] = (bike_data["end_time"] - t_min) / 60

# Introduce random permutations to make continuous
bike_data["start_time"] = bike_data["start_time"] + np.random.rand(*bike_data["start_time"].shape)
bike_data["end_time"] = bike_data["end_time"] + np.random.rand(*bike_data["end_time"].shape)

train_time = 12*7*24*60
train_bike_data = bike_data[bike_data.end_time <= train_time]

train_sorted_stations = []
for st_id in train_bike_data.end_id.sort_values().unique():
    train_sorted_stations.append(train_bike_data[train_bike_data.end_id==st_id])

train_sorted_stations[0].head()


,start_id,end_id,start_time,duration,end_time,dist
3657,667,1,503.092859,2280,541.603413,8.365682
5041,330,1,548.213552,1680,576.156181,4.997948
6023,254,1,586.376298,720,598.247826,0.286054
6570,6,1,613.162981,1260,634.696453,2.675239
7252,803,1,646.864131,1080,664.872446,2.768124


## Testing m3 likelihood recursion equation

In [53]:
beta = 0.001


def B(h, t, t_prime):
    if h == 1:
        return trueB(1, t, t_prime)

    term1 = np.exp(-1*beta*(t[h-1] - t[h-2])) * B(h-1, t, t_prime)
    # Now want number of elements in t_prime smaller than t[h-1] and than t[h-2]

    counter = 0
    for item in t_prime:
        if item > t[h-1]:
            break
        counter += 1

    upper = counter

    counter2 = 0
    for item in t_prime:
        if item > t[h-2]:
            break
        counter2 += 1

    lower = counter2 + 1

    term2 = np.sum([np.exp(-1*beta*(t[h-1] - t_prime[k-1])) for k in range(lower, upper+1)])


    return term1 + term2

def trueB(h, t, t_prime):

    counter = 0
    for item in t_prime:
        if item > t[h-1]:
            break
        counter += 1
    upper = counter

    return np.sum([np.exp(-1*beta*(t[h-1] - t_prime[k-1])) for k in range(1, upper+1)])

print(B(7, train_sorted_stations[0].start_time.to_numpy(), train_sorted_stations[0].end_time.to_numpy()))
print(trueB(7, train_sorted_stations[0].start_time.to_numpy(), train_sorted_stations[0].end_time.to_numpy()))

5.435378162783351
5.435378162783351
